#### Regression modeling of house prices in King County, Washington

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler

This notebook details construction of our regression model for predicting housing prices in King County, Washington.  First we read in the data set.

In [2]:
king = pd.read_csv('kc_house_data.csv')

In [3]:
king.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


This step trims down the dataset to the most relevant columns.  

In [4]:
king = king[['price','bedrooms','bathrooms','sqft_living','grade','sqft_lot']]
king.head()

,price,bedrooms,bathrooms,sqft_living,grade,sqft_lot
0,221900.0,3,1.00,1180,7,5650
1,538000.0,3,2.25,2570,7,7242
2,180000.0,2,1.00,770,6,10000
3,604000.0,4,3.00,1960,7,5000
4,510000.0,3,2.00,1680,8,8080


An initial correlation table can reveal those categories most correlated with price.  The top four correlations are, in descending order of correlation: sqft_living, grade, bathrooms, and bedrooms.

In [5]:
king.corr()

,price,bedrooms,bathrooms,sqft_living,grade,sqft_lot
price,1.000000,0.308787,0.525906,0.701917,0.667951,0.089876
bedrooms,0.308787,1.000000,0.514508,0.578212,0.356563,0.032471
bathrooms,0.525906,0.514508,1.000000,0.755758,0.665838,0.088373
sqft_living,0.701917,0.578212,0.755758,1.000000,0.762779,0.173453
grade,0.667951,0.356563,0.665838,0.762779,1.000000,0.114731
sqft_lot,0.089876,0.032471,0.088373,0.173453,0.114731,1.000000


Exploratory visualization reveals that square footage of the lot seems to be heavily positively skewed. 
This fact confirmed below by the skew statistic

In [6]:
print('distribution skew:' ,king['sqft_lot'].skew())
print('distribution median:',king['sqft_lot'].median())
print('distribution standard deviation:', round(king['sqft_lot'].std()))

distribution skew: 13.072603567136046
distribution median: 7618.0
distribution standard deviation: 41413


The scatter plot of lot size to price of the lot from the from the exploratory visualization seems to indicate three natural divisions of the distribution.  The first corresponds to smaller, and more expensive urban lots.  The high price of these lots reflects the densly packed and expense of urban real estate.  The second division corresponds to less expensive and larger suburban lots.  The third division represents larger and less expensive rural real estate.  We bin and parse out into dummy variables this column.  

In [7]:
bins = [0,8000, 40000, 500000]

bin_names = ['urban', 'suburban', 'rural']

king['sqft_lot'] = pd.cut(king['sqft_lot'], bins, labels = bin_names)

lot_dummies = pd.get_dummies(king.sqft_lot).iloc[:,:2]

king = pd.concat([king, lot_dummies], axis = 1)

king.drop(['sqft_lot'], axis = 1, inplace = True)

Next, we define a column that appears, from exporatory analysis, to be relevant to house price.  Occupancy per square foot represents a rough estimate of space available to an occupant.  We calculate this value by dividing square foot of living space by the number of bedrooms

In [8]:
king['occupancy_per_sqft'] = round(king['sqft_living']/king['bedrooms'])

In [9]:
king.head()

,price,bedrooms,bathrooms,sqft_living,grade,urban,suburban,occupancy_per_sqft
0,221900.0,3,1.00,1180,7,1,0,393.0
1,538000.0,3,2.25,2570,7,1,0,857.0
2,180000.0,2,1.00,770,6,0,1,385.0
3,604000.0,4,3.00,1960,7,1,0,490.0
4,510000.0,3,2.00,1680,8,0,1,560.0


One of the most highly correlated variable to house price is the grade of the structure.  We break this variable into dummy variables. 

In [10]:
king_dummies = pd.get_dummies(king.grade).iloc[:,1:]
king_gradedum = pd.concat([king,king_dummies], axis = 1)

king_final = king_gradedum.drop(['grade'], axis = 1)

king_final.head()

,price,bedrooms,bathrooms,sqft_living,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
0,221900.0,3,1.00,1180,1,0,393.0,0,0,0,1,0,0,0,0,0,0
1,538000.0,3,2.25,2570,1,0,857.0,0,0,0,1,0,0,0,0,0,0
2,180000.0,2,1.00,770,0,1,385.0,0,0,1,0,0,0,0,0,0,0
3,604000.0,4,3.00,1960,1,0,490.0,0,0,0,1,0,0,0,0,0,0
4,510000.0,3,2.00,1680,0,1,560.0,0,0,0,0,1,0,0,0,0,0


With the manipulations of the dataframe variables complete we break the dataframe into x and y variable for the regression analysis and then separate those divisions into training and testing batches.

In [11]:
king_y = pd.DataFrame(king_final.price)

king_x = king_final.drop(['price'], axis = 1)

king_train_x, king_test_x, king_train_y, king_test_y = train_test_split(
    king_x, king_y, test_size = .5, random_state = 5)
   

The head of the training set:

In [12]:
king_train_x.head()

,bedrooms,bathrooms,sqft_living,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
8991,3,3.00,3490,1,0,1163.0,0,0,0,0,0,1,0,0,0,0
16827,5,2.75,2190,0,1,438.0,0,0,0,1,0,0,0,0,0,0
19166,3,2.25,2990,0,0,997.0,0,0,0,0,0,1,0,0,0,0
8072,4,2.75,2200,0,0,550.0,0,0,0,1,0,0,0,0,0,0
16264,2,1.00,1050,1,0,525.0,0,0,1,0,0,0,0,0,0,0


The first iteration of the regression analysis will use just the moderately correlated bathrooms and bedrooms variables.

In [13]:
king_train_x_a = king_train_x[['bathrooms', 'bedrooms',]]

In [14]:
king_train_x_a.head()

,bathrooms,bedrooms
8991,3.00,3
16827,2.75,5
19166,2.25,3
8072,2.75,4
16264,1.00,2


In [15]:

model = sm.OLS(king_train_y, king_train_x_a).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.780
Model:                            OLS   Adj. R-squared (uncentered):              0.780
Method:                 Least Squares   F-statistic:                          1.913e+04
Date:                Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:09:03   Log-Likelihood:                     -1.5152e+05
No. Observations:               10798   AIC:                                  3.030e+05
Df Residuals:                   10796   BIC:                                  3.031e+05
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
bathrooms   2.267e+05   4175.616     54.284      0.000    2.18e+05    2.35e+05
bedrooms    1.717e+04   2669.706      6.431      0.000    1.19e+04    2.24e+04
==============================================================================
Omnibus:                     7934.615   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           278320.427
Skew:                           3.147   Prob(JB):                         0.00
Kurtosis:                      27.062   Cond. No.                         6.99
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R^2 is actually quite high at .773.  For the next iteration we use the metric we built by dividing living area by bedrooms.

In [16]:
king_train_x_b = king_train_x[['bathrooms', 'bedrooms', 'occupancy_per_sqft']]

In [17]:
king_train_x_b.head()

,bathrooms,bedrooms,occupancy_per_sqft
8991,3.00,3,1163.0
16827,2.75,5,438.0
19166,2.25,3,997.0
8072,2.75,4,550.0
16264,1.00,2,525.0


In [18]:

model = sm.OLS(king_train_y, king_train_x_b).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.810
Model:                            OLS   Adj. R-squared (uncentered):              0.810
Method:                 Least Squares   F-statistic:                          1.535e+04
Date:                Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:09:03   Log-Likelihood:                     -1.5072e+05
No. Observations:               10798   AIC:                                  3.015e+05
Df Residuals:                   10795   BIC:                                  3.015e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bathrooms           9.121e+04   5072.111     17.983      0.000    8.13e+04    1.01e+05
bedrooms            1.018e+04   2485.538      4.098      0.000    5312.750    1.51e+04
occupancy_per_sqft   530.0258     12.789     41.445      0.000     504.957     555.094
==============================================================================
Omnibus:                     9154.504   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           504923.838
Skew:                           3.762   Prob(JB):                         0.00
Kurtosis:                      35.644   Cond. No.                     1.31e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.31e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

A performance gain of about .025 R^2.  Next we incorporate lot size variable that we binned into urban, suburban, and rural categories and then divided into dummy categories.

In [19]:
king_train_x_c = king_train_x[['bathrooms', 'bedrooms', 'occupancy_per_sqft', 'urban', 'suburban']]

In [20]:
king_train_x_c.head()

,bathrooms,bedrooms,occupancy_per_sqft,urban,suburban
8991,3.00,3,1163.0,1,0
16827,2.75,5,438.0,0,1
19166,2.25,3,997.0,0,0
8072,2.75,4,550.0,0,0
16264,1.00,2,525.0,1,0


In [21]:

model = sm.OLS(king_train_y, king_train_x_c).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.817
Model:                            OLS   Adj. R-squared (uncentered):              0.817
Method:                 Least Squares   F-statistic:                              9620.
Date:                Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:09:03   Log-Likelihood:                     -1.5053e+05
No. Observations:               10798   AIC:                                  3.011e+05
Df Residuals:                   10793   BIC:                                  3.011e+05
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bathrooms           8.326e+04   5038.068     16.526      0.000    7.34e+04    9.31e+04
bedrooms            4.078e+04   2970.160     13.731      0.000     3.5e+04    4.66e+04
occupancy_per_sqft   613.1454     13.427     45.665      0.000     586.826     639.465
urban              -1.626e+05   8243.287    -19.720      0.000   -1.79e+05   -1.46e+05
suburban           -1.437e+05   9162.232    -15.687      0.000   -1.62e+05   -1.26e+05
==============================================================================
Omnibus:                     8648.671   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           432207.010
Skew:                           3.464   Prob(JB):                         0.00
Kurtosis:                      33.210   Cond. No.                     2.93e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.93e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

That varaible supplied a performance gain of about .01 R^2.  Next we incorporate the 'grade' variable that we divided into dummy categories.

In [22]:
king_train_x.head()

,bedrooms,bathrooms,sqft_living,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
8991,3,3.00,3490,1,0,1163.0,0,0,0,0,0,1,0,0,0,0
16827,5,2.75,2190,0,1,438.0,0,0,0,1,0,0,0,0,0,0
19166,3,2.25,2990,0,0,997.0,0,0,0,0,0,1,0,0,0,0
8072,4,2.75,2200,0,0,550.0,0,0,0,1,0,0,0,0,0,0
16264,2,1.00,1050,1,0,525.0,0,0,1,0,0,0,0,0,0,0


In [23]:

model = sm.OLS(king_train_y, king_train_x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.876
Model:                            OLS   Adj. R-squared (uncentered):              0.876
Method:                 Least Squares   F-statistic:                              4764.
Date:                Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:09:03   Log-Likelihood:                     -1.4842e+05
No. Observations:               10798   AIC:                                  2.969e+05
Df Residuals:                   10782   BIC:                                  2.970e+05
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bedrooms           -1.791e+04   5278.886     -3.392      0.001   -2.83e+04   -7559.577
bathrooms          -2.308e+04   4674.279     -4.937      0.000   -3.22e+04   -1.39e+04
sqft_living          179.7060      9.300     19.323      0.000     161.476     197.935
urban               9.938e+04   9848.008     10.091      0.000    8.01e+04    1.19e+05
suburban            5.746e+04   9800.874      5.863      0.000    3.83e+04    7.67e+04
occupancy_per_sqft    -8.0521     28.636     -0.281      0.779     -64.184      48.080
4                   5.573e+04   5.76e+04      0.968      0.333   -5.71e+04    1.69e+05
5                    6.95e+04   2.87e+04      2.423      0.015    1.33e+04    1.26e+05
6                   9.506e+04   2.15e+04      4.430      0.000     5.3e+04    1.37e+05
7                   1.224e+05   2.19e+04      5.584      0.000    7.94e+04    1.65e+05
8                   1.946e+05    2.3e+04      8.452      0.000    1.49e+05     2.4e+05
9                   3.189e+05   2.42e+04     13.157      0.000    2.71e+05    3.66e+05
10                  5.204e+05   2.62e+04     19.879      0.000    4.69e+05    5.72e+05
11                   7.63e+05   3.04e+04     25.077      0.000    7.03e+05    8.23e+05
12                  1.251e+06    4.5e+04     27.816      0.000    1.16e+06    1.34e+06
13                  2.947e+06   9.17e+04     32.126      0.000    2.77e+06    3.13e+06
==============================================================================
Omnibus:                     5690.948   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            88540.208
Skew:                           2.171   Prob(JB):                         0.00
Kurtosis:                      16.339   Cond. No.                     1.03e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.03e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now for the moment of truth.  Validating the model with the testing batch.  The head of the testing x variable as follows:

This last regression provided the strongest R^2 at .87.

In [24]:
king_test_x.head()

,bedrooms,bathrooms,sqft_living,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
15393,4,3.00,1990,1,0,498.0,0,0,0,1,0,0,0,0,0,0
6035,4,4.00,6330,0,1,1582.0,0,0,0,0,0,0,0,0,0,1
12871,4,2.75,4270,0,1,1068.0,0,0,0,0,0,0,0,1,0,0
21099,4,2.50,1950,1,0,488.0,0,0,0,0,1,0,0,0,0,0
11629,2,1.50,1150,1,0,575.0,0,0,0,1,0,0,0,0,0,0


In [25]:
model = sm.OLS(king_test_y, king_test_x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.594
Method:                 Least Squares   F-statistic:                     1052.
Date:                Thu, 22 Oct 2020   Prob (F-statistic):               0.00
Time:                        11:09:03   Log-Likelihood:            -1.4928e+05
No. Observations:               10799   AIC:                         2.986e+05
Df Residuals:                   10783   BIC:                         2.987e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bedrooms           -5.319e+04   7566.303     -7.030      0.000    -6.8e+04   -3.84e+04
bathrooms          -8043.8107   5002.380     -1.608      0.108   -1.78e+04    1761.775
sqft_living          242.7644     12.140     19.996      0.000     218.967     266.562
urban               1.005e+05    1.1e+04      9.127      0.000    7.89e+04    1.22e+05
suburban            6.449e+04    1.1e+04      5.869      0.000     4.3e+04     8.6e+04
occupancy_per_sqft  -133.6436     37.885     -3.528      0.000    -207.905     -59.382
4                     1.5e+05   8.09e+04      1.853      0.064   -8657.666    3.09e+05
5                   1.253e+05   3.34e+04      3.751      0.000    5.98e+04    1.91e+05
6                   1.351e+05   2.76e+04      4.902      0.000    8.11e+04    1.89e+05
7                   1.696e+05   2.84e+04      5.970      0.000    1.14e+05    2.25e+05
8                   2.191e+05   2.94e+04      7.441      0.000    1.61e+05    2.77e+05
9                   3.281e+05   3.03e+04     10.820      0.000    2.69e+05    3.88e+05
10                  5.137e+05   3.16e+04     16.259      0.000    4.52e+05    5.76e+05
11                  8.087e+05   3.47e+04     23.285      0.000    7.41e+05    8.77e+05
12                  1.341e+06   4.69e+04     28.578      0.000    1.25e+06    1.43e+06
13                  1.878e+06   1.05e+05     17.838      0.000    1.67e+06    2.08e+06
==============================================================================
Omnibus:                     7289.689   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           308053.934
Skew:                           2.695   Prob(JB):                         0.00
Kurtosis:                      28.604   Cond. No.                     1.12e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.12e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

R^2 = .610 These results were disapointing.   R^2 value of this model was significantly reduced from the previous regression.  The P value of the 'bathrooms' variable is decreased to irrelevance over past regressions.  Occupancy per square foot also saw a strange drop in P value.

In [26]:
king_test_x_a = king_test_x.drop(['bathrooms', 'sqft_living'], axis = 1)

In [27]:
model = sm.OLS(king_test_y, king_test_x_a).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     1140.
Date:                Thu, 22 Oct 2020   Prob (F-statistic):               0.00
Time:                        11:09:03   Log-Likelihood:            -1.4948e+05
No. Observations:               10799   AIC:                         2.990e+05
Df Residuals:                   10785   BIC:                         2.991e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bedrooms            8.502e+04   3082.518     27.580      0.000     7.9e+04    9.11e+04
urban               9.199e+04   1.12e+04      8.232      0.000    7.01e+04    1.14e+05
suburban              5.9e+04   1.12e+04      5.274      0.000    3.71e+04    8.09e+04
occupancy_per_sqft   562.0349     15.758     35.668      0.000     531.147     592.923
4                  -2.372e+05   8.01e+04     -2.962      0.003   -3.94e+05   -8.02e+04
5                  -2.785e+05   2.72e+04    -10.229      0.000   -3.32e+05   -2.25e+05
6                  -2.717e+05   1.91e+04    -14.209      0.000   -3.09e+05   -2.34e+05
7                   -2.47e+05   1.98e+04    -12.477      0.000   -2.86e+05   -2.08e+05
8                  -1.893e+05   2.15e+04     -8.787      0.000   -2.32e+05   -1.47e+05
9                  -4.852e+04   2.41e+04     -2.011      0.044   -9.58e+04   -1218.667
10                  1.693e+05   2.68e+04      6.311      0.000    1.17e+05    2.22e+05
11                  5.331e+05   3.22e+04     16.530      0.000     4.7e+05    5.96e+05
12                  1.155e+06   4.67e+04     24.738      0.000    1.06e+06    1.25e+06
13                  1.892e+06   1.07e+05     17.659      0.000    1.68e+06     2.1e+06
==============================================================================
Omnibus:                     8350.685   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           566327.087
Skew:                           3.162   Prob(JB):                         0.00
Kurtosis:                      37.909   Cond. No.                     3.04e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
king_test_x_a.head()

,bedrooms,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
15393,4,1,0,498.0,0,0,0,1,0,0,0,0,0,0
6035,4,0,1,1582.0,0,0,0,0,0,0,0,0,0,1
12871,4,0,1,1068.0,0,0,0,0,0,0,0,1,0,0
21099,4,1,0,488.0,0,0,0,0,1,0,0,0,0,0
11629,2,1,0,575.0,0,0,0,1,0,0,0,0,0,0
